In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# 1. 路径定义
edges_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_physical_base.csv"
nodes_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive_physical_base.csv"

# 2. 加载物理底座数据 (锁定 ID 为字符串)
df_edges = pd.read_csv(edges_path, dtype={'u': str, 'v': str})
df_nodes = pd.read_csv(nodes_path, dtype={'node_id': str})

# 3. 构建节点坐标快速查找表 (ID -> (x, y))
node_coords = df_nodes.set_index('node_id')[['x', 'y']].to_dict('index')

print(">>> 正在计算路网中点并构建 KDTree 索引...")

# 4. 计算每条边的几何中点
midpoints = []
edge_map = [] # 用于记录索引与边的对应关系

for idx, row in df_edges.iterrows():
    u, v = row['u'], row['v']
    
    # 提取端点坐标
    coord_u = node_coords[u]
    coord_v = node_coords[v]
    
    # 计算几何中点 (Arithmetic Mean) 
    mid_x = (coord_u['x'] + coord_v['x']) / 2
    mid_y = (coord_u['y'] + coord_v['y']) / 2
    
    midpoints.append([mid_x, mid_y])
    edge_map.append({'u': u, 'v': v, 'index': idx})

# 5. 构建 KDTree 空间索引 [cite: 23]
midpoints_array = np.array(midpoints)
spatial_index = KDTree(midpoints_array)

# 6. 将中点坐标存回边表 (用于可视化验证)
df_edges['mid_x'] = midpoints_array[:, 0]
df_edges['mid_y'] = midpoints_array[:, 1]

# 7. 体检报告 (Health Check)
print("-" * 40)
print(f"KDTree 索引构建成功！")
print(f"索引节点规模: {len(midpoints)} 个边中点")
print(f"坐标维度: {midpoints_array.shape[1]}D")
print("-" * 40)

# 8. 输出中间文件供后续步骤使用
output_dir = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
output_path = f"{output_dir}\\edges_with_midpoints.csv"
df_edges.to_csv(output_path, index=False)
print(f"包含中点信息的边表已保存: {output_path}")

>>> 正在计算路网中点并构建 KDTree 索引...
----------------------------------------
KDTree 索引构建成功！
索引节点规模: 89655 个边中点
坐标维度: 2D
----------------------------------------
包含中点信息的边表已保存: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_with_midpoints.csv


In [4]:
import pandas as pd
import numpy as np
import re
from scipy.spatial import KDTree

# 1. 路径定义 (基于您的绝对路径)
base_dir = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
nodes_path = f"{base_dir}\\nodes_drive_physical_base.csv"
edges_mid_path = f"{base_dir}\\edges_with_midpoints.csv"
aadt_path = f"{base_dir}\\MDOT_SHA_Annual_Average_Daily_Traffic_Baltimore.csv"

# 2. 加载数据集
df_nodes = pd.read_csv(nodes_path, dtype={'node_id': str})
df_edges = pd.read_csv(edges_mid_path, dtype={'u': str, 'v': str})
df_aadt = pd.read_csv(aadt_path)

# 3. 流量监测点坐标重构 (Coordinate Reconstruction)
def extract_id(s):
    if pd.isna(s): return None
    match = re.search(r'(\d+)', str(s)) # 提取 {} 中的数字
    return match.group(1) if match else None

df_aadt['extracted_node_id'] = df_aadt['node start'].apply(extract_id)

# 通过节点表补齐经纬度
df_aadt_coords = df_aadt.merge(
    df_nodes[['node_id', 'x', 'y']], 
    left_on='extracted_node_id', 
    right_on='node_id', 
    how='inner'
)

print(f">>> 成功重构坐标的监测点数量: {len(df_aadt_coords)}")

# 4. 构建 KDTree 并执行空间吸附
# 提取路网边中点坐标
edge_midpoints = df_edges[['mid_x', 'mid_y']].values
spatial_tree = KDTree(edge_midpoints)

# 检索监测点最近的边索引
aadt_points = df_aadt_coords[['x', 'y']].values
distances, indices = spatial_tree.query(aadt_points)

# 5. 冲突处理：峰值捕捉 (MAX Strategy)
# 使用 'AADT (Current)' 作为流量字段
volume_col = 'AADT (Current)'
df_aadt_coords['matched_edge_idx'] = indices

# 按边索引聚合，取最大流量
edge_volume_agg = df_aadt_coords.groupby('matched_edge_idx')[volume_col].max().to_dict()

# 6. 将流量属性注入边表
df_edges['volume'] = np.nan
for edge_idx, vol in edge_volume_agg.items():
    df_edges.at[edge_idx, 'volume'] = vol

# 7. 体检报告 (Data Audit)
matched_edges = df_edges['volume'].notna().sum()
print("-" * 45)
print(f"流量吸附完成！")
print(f"路网匹配覆盖率: {matched_edges / len(df_edges):.2%}")
print(f"全局最高流量 (Peak): {df_edges['volume'].max():.0f}")
print("-" * 45)

# 8. 保存输出
output_path = f"{base_dir}\\edges_with_volume.csv"
df_edges.to_csv(output_path, index=False)
print(f"已生成包含流量属性的边表: {output_path}")

>>> 成功重构坐标的监测点数量: 855
---------------------------------------------
流量吸附完成！
路网匹配覆盖率: 0.82%
全局最高流量 (Peak): 134122
---------------------------------------------
已生成包含流量属性的边表: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_with_volume.csv


In [8]:
import pandas as pd
import numpy as np
import re
import os
from scipy.spatial import KDTree

# 1. 设置绝对路径
base_dir = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
nodes_path = os.path.join(base_dir, "nodes_drive_physical_base.csv")
edges_path = os.path.join(base_dir, "edges_with_midpoints.csv")
aadt_path = os.path.join(base_dir, "MDOT_SHA_Annual_Average_Daily_Traffic_Baltimore.csv")

# 2. 加载数据并锁定 ID 类型
print(">>> 正在加载数据集...")
nodes_df = pd.read_csv(nodes_path, dtype={'node_id': str})
edges_df = pd.read_csv(edges_path, dtype={'u': str, 'v': str})
aadt_df = pd.read_csv(aadt_path)

# 3. 增强型坐标提取与审计逻辑
def extract_all_ids(s):
    if pd.isna(s): return []
    return re.findall(r'(\d+)', str(s))

# 构建节点快速索引表 {id: (x, y)}
nodes_lookup = nodes_df.set_index('node_id')[['x', 'y']].to_dict('index')
nodes_lookup = {k: (v['x'], v['y']) for k, v in nodes_lookup.items()}

def get_best_coord(row):
    # 同时扫描 'node start' 和 'node(s) end' 列
    ids = extract_all_ids(row['node start']) + extract_all_ids(row['node(s) end'])
    for node_id in ids:
        if node_id in nodes_lookup:
            return nodes_lookup[node_id]
    return None, None

print(">>> 正在执行监测点地理坐标重构...")
# 执行坐标恢复
coords = aadt_df.apply(get_best_coord, axis=1)
aadt_df['x'], aadt_df['y'] = zip(*[c if c else (None, None) for c in coords])

# 数据审计：为何有些点无法匹配？
total_rows = len(aadt_df)
no_node_info = aadt_df[aadt_df['node start'].isna() & aadt_df['node(s) end'].isna()]
aadt_mappable = aadt_df.dropna(subset=['x', 'y']).copy()

# 4. 执行 KDTree 空间吸附 (Snapping)
# 只有恢复了坐标的点才能参与吸附
print(f">>> 正在对 {len(aadt_mappable)} 个有效监测点执行 KDTree 空间吸附...")
tree = KDTree(edges_df[['mid_x', 'mid_y']].values)
distances, indices = tree.query(aadt_mappable[['x', 'y']].values)

# 5. 流量注入与冲突处理 (MAX 策略)
aadt_mappable['edge_idx'] = indices
# 使用 AADT (Current) 作为核心流量指标
volume_map = aadt_mappable.groupby('edge_idx')['AADT (Current)'].max().to_dict()

edges_df['volume'] = np.nan
for idx, vol in volume_map.items():
    edges_df.at[idx, 'volume'] = vol

# 6. 最终质量审计报告 (Data Audit Report)
print("-" * 50)
print(f"【阶段 3.2 流量吸附审计报告】")
print(f"1. AADT 原始记录总数: {total_rows}")
print(f"2. 原始 CSV 缺失 node 标识的记录数: {len(no_node_info)} (约 {len(no_node_info)/total_rows:.1%})")
print(f"3. 成功恢复物理坐标并参与吸附的记录: {len(aadt_mappable)}")
print(f"4. 流量吸附覆盖的边数: {edges_df['volume'].notna().sum()}")
print(f"5. 全网最大监测流量: {edges_df['volume'].max():.0f}")
print("-" * 50)

# 7. 保存结果
output_path = os.path.join(base_dir, "edges_with_volume.csv")
edges_df.to_csv(output_path, index=False)
print(f">>> 已生成包含流量属性的边表: {output_path}")

>>> 正在加载数据集...
>>> 正在执行监测点地理坐标重构...
>>> 正在对 1102 个有效监测点执行 KDTree 空间吸附...
--------------------------------------------------
【阶段 3.2 流量吸附审计报告】
1. AADT 原始记录总数: 2398
2. 原始 CSV 缺失 node 标识的记录数: 1295 (约 54.0%)
3. 成功恢复物理坐标并参与吸附的记录: 1102
4. 流量吸附覆盖的边数: 853
5. 全网最大监测流量: 165202
--------------------------------------------------
>>> 已生成包含流量属性的边表: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_with_volume.csv


In [9]:
import pandas as pd
import numpy as np
import os

# 1. 设置路径
base_dir = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
edges_volume_path = os.path.join(base_dir, "edges_with_volume.csv")

# 2. 加载数据
df_edges = pd.read_csv(edges_volume_path)

print(">>> 正在执行全局阻抗更新与属性补全...")

# 3. 统计学防御：填补“数据黑洞” (Median Imputation)
# 计算已匹配路段流量的中位数作为全网“背景底噪”
median_volume = df_edges['volume'].median()
# 如果中位数获取失败（如全空），设定一个保守值（如 3000 AADT）
if np.isnan(median_volume): median_volume = 3000.0

print(f"统计结果：已匹配路段流量中位数 = {median_volume:.0f} AADT")

# 执行填补：将 NaN 流量替换为中位数
df_edges['volume_filled'] = df_edges['volume'].fillna(median_volume)

# 4. 通行能力建模 (Capacity Modeling)
# 假设车道数：优先使用原始数据车道（如果存在），否则默认为 2 车道
# 注：如果之前的步骤没有提取 lanes，此处默认支路为 2 车道，主干道为 4 车道（根据流量判定）
def estimate_lanes(row):
    if row['volume_filled'] > 50000: return 4 # 判定为高速/主干
    return 2 # 判定为普通支路

df_edges['lanes'] = df_edges.apply(estimate_lanes, axis=1)

# 计算每日通行能力 (Capacity)
# 经验公式：单车道每日设计容量约为 15,000 辆
df_edges['capacity'] = df_edges['lanes'] * 15000

# 5. 激活 BPR 阻抗机理 (BPR Resistance Function)
# 公式：T = T0 * (1 + alpha * (V/C)^beta)
alpha = 0.15
beta = 4

print(">>> 正在基于 BPR 函数计算动态通行阻抗...")
df_edges['free_flow_time'] = df_edges['travel_time'] # 步骤 2 计算的原始时间

# 计算考虑拥堵后的最终通行时间 (Weight)
df_edges['final_weight'] = df_edges['free_flow_time'] * (
    1 + alpha * (df_edges['volume_filled'] / df_edges['capacity'])**beta
)

# 6. 体检报告 (Final Audit)
congestion_increase = (df_edges['final_weight'].mean() - df_edges['free_flow_time'].mean()) / df_edges['free_flow_time'].mean()
print("-" * 50)
print(f"【阶段 3.3 阻抗建模审计报告】")
print(f"1. 全局背景流量填补值: {median_volume:.0f} AADT")
print(f"2. 全网平均自由流时间: {df_edges['free_flow_time'].mean():.2f} s")
print(f"3. 全网平均动态阻抗时间: {df_edges['final_weight'].mean():.2f} s")
print(f"4. 初始拥堵成本增加比: {congestion_increase:.2%}")
print("-" * 50)

# 7. 持久化输出
output_path = os.path.join(base_dir, "edges_final_model.csv")
df_edges.to_csv(output_path, index=False)
print(f">>> 最终仿真模型已保存: {output_path}")

>>> 正在执行全局阻抗更新与属性补全...
统计结果：已匹配路段流量中位数 = 5255 AADT
>>> 正在基于 BPR 函数计算动态通行阻抗...
--------------------------------------------------
【阶段 3.3 阻抗建模审计报告】
1. 全局背景流量填补值: 5255 AADT
2. 全网平均自由流时间: 15.16 s
3. 全网平均动态阻抗时间: 15.17 s
4. 初始拥堵成本增加比: 0.03%
--------------------------------------------------
>>> 最终仿真模型已保存: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_final_model.csv
